In [ ]:
import azureml.core
import pandas as pd
import numpy as np
import logging

print("Azure ML SDK Version:",azureml.core.VERSION)

In [ ]:
from azureml.core import Workspace, Experiment

#use json config file to access remote vscode
ws=Workspace.from_config()
#create experiment
exp = Experiment(ws, "udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
run = exp.start_logging()

In [ ]:
rom azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
amlcompute_cluster_name = "cpu-cluster"

provisioning_config=AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4)

compute_target=ComputeTarget.create(ws,amlcompute_cluster_name,provisioning_config)

compute_target.wait_for_completion(show_output=True,min_node_count=None,timeout_in_minutes=20)

# Optimize discrete hyperparameters:
* -C        = Inverse of regularization strength. Smaller values cause stronger regularization
* --max_iter= Maximum number of iterations to converge

Use Random Sampling

[Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters) for hyperparameters configuration

[Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn) to train scikit-learn models at azure

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig


In [ ]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

In [ ]:
from azureml.core import Environment
from azureml.core import ScriptRunConfig
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = 'conda_dependencies.yml')
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

In [ ]:
# Specify parameter sampler
ps = RandomParameterSampling( {
        "-C": choice(0.05, 0.5, 1, 2, 4),
        "--max_iter": choice(5,10,50,100)        
    }
)


# Specify a Policy
# (using early termination policy the Bayesian Sampling is not supported)
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)


if "training" not in os.listdir():
    os.mkdir("./training")


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.


hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name="accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
#run experiment
hyperdrive_run = exp.submit(hyperdrive_config)
#visualize experiment

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)


In [ ]:
import joblib
# Get your best run and save the model from that run.
### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###